In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow
tensorflow.keras.__version__
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from numpy.random import seed
seed(1)

import warnings
warnings.simplefilter('ignore')

In [2]:
df=pd.read_csv("../Data/stock_data.csv")
df.head()

,Date,open,high,low,close,volume,dividends,stock_splits,price_change_$,price_change_%,price_swing,volume_change,volume_change_%,price_change_binary
0,1993-02-01,26.098205,26.265144,26.098205,26.265144,480500,0.0,0,0.185486,0.711228,0.166939,-522700.0,-52.103270,1.0
1,1993-02-02,26.246586,26.339331,26.190940,26.320782,201300,0.0,0,0.055637,0.211830,0.148391,-279200.0,-58.106139,1.0
2,1993-02-03,26.357880,26.617563,26.339331,26.599014,529400,0.0,0,0.278233,1.057083,0.278232,328100.0,162.990561,1.0
3,1993-02-04,26.691763,26.765958,26.394982,26.710312,531500,0.0,0,0.111298,0.418428,0.370977,2100.0,0.396675,1.0
4,1993-02-05,26.691771,26.747417,26.543380,26.691771,492100,0.0,0,-0.018541,-0.069416,0.204037,-39400.0,-7.412982,0.0


## Pre-processing the Data

In [13]:
X=df[["close", "volume_change_%"]]
y=df[["price_change_binary"]]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [5]:
# # Use when units and scale are the same
# X_scaler = MinMaxScaler().fit(X_train)
# y_scaler = MinMaxScaler().fit(y_train)

# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

In [15]:
# Use when units are different
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [16]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

## Creating Model

In [17]:
# Create model and add layers
model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=len(X.columns)))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='sigmoid'))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               300       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 10,602
Trainable params: 10,602
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=50,
    shuffle=True,
    verbose=2
)

Epoch 1/50
165/165 - 0s - loss: 0.6828 - accuracy: 0.5524
Epoch 2/50
165/165 - 0s - loss: 0.6738 - accuracy: 0.5823
Epoch 3/50
165/165 - 0s - loss: 0.6715 - accuracy: 0.5876
Epoch 4/50
165/165 - 0s - loss: 0.6701 - accuracy: 0.5848
Epoch 5/50
165/165 - 0s - loss: 0.6698 - accuracy: 0.5836
Epoch 6/50
165/165 - 0s - loss: 0.6687 - accuracy: 0.5884
Epoch 7/50
165/165 - 0s - loss: 0.6695 - accuracy: 0.5895
Epoch 8/50
165/165 - 0s - loss: 0.6685 - accuracy: 0.5893
Epoch 9/50
165/165 - 0s - loss: 0.6683 - accuracy: 0.5840
Epoch 10/50
165/165 - 0s - loss: 0.6689 - accuracy: 0.5884
Epoch 11/50
165/165 - 0s - loss: 0.6675 - accuracy: 0.5918
Epoch 12/50
165/165 - 0s - loss: 0.6682 - accuracy: 0.5891
Epoch 13/50
165/165 - 0s - loss: 0.6675 - accuracy: 0.5880
Epoch 14/50
165/165 - 0s - loss: 0.6676 - accuracy: 0.5897
Epoch 15/50
165/165 - 0s - loss: 0.6675 - accuracy: 0.5891
Epoch 16/50
165/165 - 0s - loss: 0.6673 - accuracy: 0.5901
Epoch 17/50
165/165 - 0s - loss: 0.6681 - accuracy: 0.5899
Epoch 

## Validating the Model

In [21]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.6780 - accuracy: 0.5691
Normal Neural Network - Loss: 0.6779671311378479, Accuracy: 0.5691428780555725
